In [0]:
# no Databricks notebook cell
import requests
import json
import time
from datetime import datetime, timezone
import math

# Spark imports
from pyspark.sql import functions as F
from pyspark.sql.types import DoubleType, LongType, StructType, StructField, StringType, TimestampType
from pyspark.sql import Window


In [0]:
import requests
import pandas as pd

# 🔑 Seu token da SPTrans
TOKEN = "725dfd912a4526086c542731327cfd72341ad64352761c16aad6ff954fa4bbb1"

# 🔐 Autenticação na API
auth = requests.post(f"https://api.olhovivo.sptrans.com.br/v2.1/Login/Autenticar?token={TOKEN}")
if auth.status_code != 200:
    raise Exception("Falha na autenticação. Verifique o token.")

# Regiões e principais bairros
regioes = {
    "Zona Leste": ["itaquera", "penha", "artur alvim", "sapopemba", "são mateus"],
    "Zona Sul": ["santo amaro", "grajaú", "capão redondo", "jabaquara", "socorro"],
    "Zona Norte": ["tucuruvi", "jaçanã", "santana", "cachoeirinha", "mandaqui"],
    "Zona Oeste": ["lapa", "butantã", "vila leopoldina", "morumbi", "perdizes"],
    "Centro": ["sé", "república", "brás", "bela vista", "liberdade"]
}

# Buscar linhas por região
todas_linhas = []
for regiao, bairros in regioes.items():
    for bairro in bairros:
        url = f"https://api.olhovivo.sptrans.com.br/v2.1/Linha/Buscar?termosBusca={bairro}"
        resp = requests.get(url, cookies=auth.cookies)
        if resp.status_code == 200:
            try:
                linhas = resp.json()
                for linha in linhas:
                    todas_linhas.append({
                        "regiao": regiao,
                        "bairro_busca": bairro,
                        "codigo_linha": linha["cl"],
                        "sentido": linha["sl"],
                        "numero_linha": linha["lt"],
                        "letra_linha": linha["tl"],
                        "origem": linha["tp"],
                        "destino": linha["ts"]
                    })
            except Exception as e:
                print(f"Erro ao processar bairro {bairro}: {e}")
        else:
            print(f"Erro na requisição para {bairro}: {resp.status_code}")

# Montar DataFrame
df = pd.DataFrame(todas_linhas).drop_duplicates(subset=["codigo_linha", "sentido"])



In [0]:
df_spark = spark.createDataFrame(df)

In [0]:
df_spark.write.mode("overwrite").saveAsTable("sp_trans2.bronze.api_linhas")